# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

In [19]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

URL = 'https://habr.com/ru/all/'
res = requests.get(URL)

KEYWORDS = ['python', 'работа', 'игры']

soup = BeautifulSoup(res.text, 'html.parser')

data_dict = {
    'date': [],
    'title': [],
    'link': [],
}

In [20]:
def get_url_data(url, page):
    if page != 1:
        url = url + '/page' + str(page)
    res = requests.get(url)
    time.sleep(0.3)
    soup = BeautifulSoup(res.text, 'html.parser')
    posts = soup.find_all('article', class_='post post_preview')
    for post in posts:
        hubs = post.find_all('a', class_='post__title_link')
        post_time = post.find('span', class_='post__time').text
        for hub in hubs:
            hub_lower = hub.text.lower()
            for keyword in KEYWORDS:
                if keyword in hub_lower:
                    title_element = post.find('a', class_='post__title_link')
                    return post_time, title_element.text, title_element.attrs.get('href')

In [22]:
for page in range(1,10):
    if get_url_data(URL, page):
        print(f'Нашлось на станице новостей №{page}')
        date, title, link = get_url_data(URL, page)
        data_dict['date'].append(date)
        data_dict['title'].append(title)
        data_dict['link'].append(link)

Нашлось на станице новостей №1
Нашлось на станице новостей №2
Нашлось на станице новостей №4
Нашлось на станице новостей №5


In [23]:
filtered_news = pd.DataFrame(data_dict)
filtered_news

,date,title,link
0,сегодня в 18:44,"Создание арт-объектов, игр и много чего ещё с ...",https://habr.com/ru/company/skillfactory/blog/...
1,сегодня в 14:58,Как мы визуальный конструктор обучения цифровы...,https://habr.com/ru/company/neuronet/blog/548934/
2,сегодня в 10:01,Как этот год работала наша внутренняя служба г...,https://habr.com/ru/company/croc/blog/548768/
3,вчера в 18:00,Работаем с lightsquid или как сделать индивиду...,https://habr.com/ru/post/548782/
4,сегодня в 18:44,"Создание арт-объектов, игр и много чего ещё с ...",https://habr.com/ru/company/skillfactory/blog/...
5,сегодня в 14:58,Как мы визуальный конструктор обучения цифровы...,https://habr.com/ru/company/neuronet/blog/548934/
6,сегодня в 10:01,Как этот год работала наша внутренняя служба г...,https://habr.com/ru/company/croc/blog/548768/
7,вчера в 18:00,Работаем с lightsquid или как сделать индивиду...,https://habr.com/ru/post/548782/


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<почта> - <дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [25]:
import json

params = {
        'emailAddresses': ["yy@ya.ru","xx@ya.ru", "12@wwwww.ru"]
    }

URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

headers = {
    'Host': 'identityprotection.avast.com',
    'Connection': 'keep-alive',
    'Content-Length': '34',
    'sec-ch-ua': '"Google Chrome";v="87", " Not;A Brand";v="99", "Chromium";v="87"',
    'Accept': 'application/json, text/plain, */*',
    'Vaar-Version': '0',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Content-Type': 'application/json;charset=UTF-8',
    'Origin': 'https://www.avast.com',
    'Sec-Fetch-Site': 'same-site',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://www.avast.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7'
}



res = requests.post(URL, headers=headers, json=params)
res

<Response [400]>

<Response [200]>

In [26]:
breaches = json.loads(res.text)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [19]:
breaches['data']

{'fc-zenit.ru': {'yy@ya.ru': [{'breachId': 3701,
    'usernameBreached': True,
    'passwordBreached': True}]},
 'cfire.mail.ru': {'yy@ya.ru': [{'breachId': 3164,
    'usernameBreached': True,
    'passwordBreached': True}]},
 'storelp.ru': {'xx@ya.ru': [{'breachId': 14357,
    'usernameBreached': True,
    'passwordBreached': True}]},
 'vk.com': {'yy@ya.ru': [{'breachId': 12,
    'usernameBreached': True,
    'passwordBreached': True}],
  'xx@ya.ru': [{'breachId': 12,
    'usernameBreached': True,
    'passwordBreached': True}]}}

In [20]:
email_leak_dict = {
    'email': [],
    'leak_date': [],
    'leak_source': [],
    'leak_title': [],
}

In [21]:
for key, value in breaches['summary'].items():
    print(key, value)

yy@ya.ru {'breaches': [12, 3164, 3701]}
xx@ya.ru {'breaches': [14357, 12]}
12@wwwww.ru {'breaches': []}


In [22]:
for key, value in breaches['summary'].items():
    email = key
    if not value['breaches']:
        email_leak_dict['email'].append(email)
        email_leak_dict['leak_date'].append('no breach')
        email_leak_dict['leak_source'].append('no breach')
        email_leak_dict['leak_title'].append('no breach')
    for breach in value['breaches']:
        leak_date = breaches['breaches'][str(breach)]['publishDate']
        leak_source = breaches['breaches'][str(breach)]['site']
        leak_title = breaches['breaches'][str(breach)]['description']
        email_leak_dict['email'].append(email)
        email_leak_dict['leak_date'].append(leak_date)
        email_leak_dict['leak_source'].append(leak_source)
        email_leak_dict['leak_title'].append(leak_title)

In [23]:
email_leak_dict

{'email': ['yy@ya.ru',
  'yy@ya.ru',
  'yy@ya.ru',
  'xx@ya.ru',
  'xx@ya.ru',
  '12@wwwww.ru'],
 'leak_date': ['2016-10-29T00:00:00Z',
  '2017-02-14T00:00:00Z',
  '2017-03-31T00:00:00Z',
  '2018-06-19T00:00:00Z',
  '2016-10-29T00:00:00Z',
  'no breach'],
 'leak_source': ['vk.com',
  'cfire.mail.ru',
  'fc-zenit.ru',
  'storelp.ru',
  'vk.com',
  'no breach'],
 'leak_title': ["Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million clear-text passwords were compromised in the breach. Breached credential sets included victims' e-mail addresses, passwords, dates of birth, phone numbers and location details. The credential set was advertised on a dark web marketplace as of June 2016 for a price of one bitcoin. ",
  "In July and August of 2016, two criminals carried out attacks on three separate forums hosted by Mail.ru, including CFire. The hackers used known SQL injection vulnerabilities found in older vBulletin forum software to obtain access to 

In [24]:
email_leak = pd.DataFrame(email_leak_dict)
email_leak

,email,leak_date,leak_source,leak_title
0,yy@ya.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
1,yy@ya.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
2,yy@ya.ru,2017-03-31T00:00:00Z,fc-zenit.ru,"In July 2010, FC Zenit's user database was all..."
3,xx@ya.ru,2018-06-19T00:00:00Z,storelp.ru,"At an unconfirmed date, StoreLP.ru's database ..."
4,xx@ya.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
5,12@wwwww.ru,no breach,no breach,no breach


# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

In [4]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

URL = 'https://habr.com/ru/all/'
res = requests.get(URL)

KEYWORDS = ['python', 'парсинг']

soup = BeautifulSoup(res.text, 'html.parser')

posts = soup.find_all('article', class_='post post_preview')


In [5]:
def get_url_data(url, page):
    if page != 1:
        url = url + '/page' + str(page)
    res = requests.get(url)
    time.sleep(0.3)
    soup = BeautifulSoup(res.text, 'html.parser')
    posts = soup.find_all('article', class_='post post_preview')
    for post in posts:
        hubs = post.find_all('a', class_='post__title_link')
        post_time = post.find('span', class_='post__time').text
        for hub in hubs:
            hub_lower = hub.text.lower()
            for keyword in KEYWORDS:
                if keyword in hub_lower:
                    title_element = post.find('a', class_='post__title_link')
                    return post_time, title_element.text, title_element.attrs.get('href')

In [6]:
data_dict = {
    'date': [],
    'title': [],
    'link': [],
}

In [7]:
for page in range(1,10):
    if get_url_data(URL, page):
        print(f'Нашлось на станице новостей №{page}')
        date, title, link = get_url_data(URL, page)
        data_dict['date'].append(date)
        data_dict['title'].append(title)
        data_dict['link'].append(link)

Нашлось на станице новостей №1


In [8]:
filtered_news = pd.DataFrame(data_dict)
filtered_news

,date,title,link
0,сегодня в 18:44,"Создание арт-объектов, игр и много чего ещё с ...",https://habr.com/ru/company/skillfactory/blog/...
